In [1]:
import numpy as np
import csv
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Dense, Lambda, Dropout
from tensorflow.keras import Model, Input
from tensorflow.keras.initializers import RandomUniform, RandomNormal


In [2]:
def vandermonde_multiplier(dim_x, m):
    dim_v = (m + 1)**dim_x
    
    v_mult_row = np.zeros((dim_x,))
    v_mult = np.zeros((dim_v, dim_x))
    
    for i_row in range(1, dim_v):
        v_mult_row[0] += 1
        
        for i_dim in range(dim_x - 1):
            if v_mult_row[i_dim] >= (m + 1):
                v_mult_row[i_dim + 1] += v_mult_row[i_dim] // (m + 1)
                v_mult_row[i_dim] %= (m + 1)
        
        v_mult[i_row, :] = v_mult_row
        
    return v_mult


In [3]:
def get_edge_list_from_file(file_path, file_name):
    edges = []
    with open(file_path + '/' + file_name) as data_file:
        data_reader = csv.reader(data_file, delimiter='\t')
        
        n_user = 0
        n_item = 0
        for row in data_reader:
            user, item, value = int(row[0]), int(row[1]), float(row[2])
            edges += [( user, item, value )]
            
            n_user = max([n_user, user])
            n_item = max([n_item, item])

    return edges, n_user, n_item


def get_observed_dic_from_edge_list(edges):
    users_dic = {}
    items_dic = {}
    user_item_dic = {}
    for user, item, value in edges:
        
        user_item_dic[(user, item)] = value

        # update users_dic
        if user in users_dic:
            users_dic[user] += [item]
        else:
            users_dic[user] = [item]

        # update items_dic
        if item in items_dic:
            items_dic[item] += [user]
        else:
            items_dic[item] = [user]
            

    return users_dic, items_dic, user_item_dic


def get_input_output(n_item, n_user, user_item_dic):
    x_one_hot = np.eye(n_item)
    
    s_full = np.ones((n_item, n_user))*-1
    for user_item, val in user_item_dic.items():
        user, item = user_item
        s_full[item - 1, user - 1] = val
        
    return x_one_hot, s_full

In [4]:
load_path = '../smooth-reconstruction-of-preference-function/data/ml-100k'

_, n_user, n_item = get_edge_list_from_file(load_path, 'u.data')
edges_tr, _, _ = get_edge_list_from_file(load_path, 'u3.base')
edges_te, _, _ = get_edge_list_from_file(load_path, 'u3.test')

users_dic_tr, items_dic_tr, r_tr = get_observed_dic_from_edge_list(edges_tr)
users_dic_te, items_dic_te, r_te = get_observed_dic_from_edge_list(edges_te)


In [5]:
x_one_hot_tr, s_full_tr = get_input_output(n_item, n_user, r_tr)
x_one_hot_te, s_full_te = get_input_output(n_item, n_user, r_te)


In [6]:
dim_x = 5
m = 5

v_mult = vandermonde_multiplier(dim_x, m)

dim_v = v_mult.shape[0]

print(dim_v)


7776


In [7]:
input_one_hot_layer = Input(shape=(n_item,))
x_layer = Dense(dim_x,
                use_bias=False,
                trainable=True,
                kernel_initializer=RandomUniform(minval=-1, maxval=1),
                kernel_constraint='non_neg',
                name='x')(input_one_hot_layer)

v_mult_layer = Dense(dim_v,
                     use_bias=False,
                     trainable=False,
                     name='v_mult')(x_layer)

v_cos_layer = Lambda(lambda x: K.cos(np.pi*x))(v_mult_layer)

drp_v_layer = Dropout(0.3)(v_cos_layer)

a_layer = Dense(n_user,
                use_bias=False,
                kernel_initializer=RandomNormal(mean=0, stddev=1e-3),
                name='a')(drp_v_layer)

mdl = Model(inputs=input_one_hot_layer, outputs=a_layer)
mdl.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1682)]            0         
_________________________________________________________________
x (Dense)                    (None, 5)                 8410      
_________________________________________________________________
v_mult (Dense)               (None, 7776)              38880     
_________________________________________________________________
lambda (Lambda)              (None, 7776)              0         
_________________________________________________________________
dropout (Dropout)            (None, 7776)              0         
_________________________________________________________________
a (Dense)                    (None, 943)               7332768   
Total params: 7,380,058
Trainable params: 7,341,178
Non-trainable params: 38,880
_______________________________________

In [8]:
# Set v_layer weights
mdl.get_layer(name='v_mult').set_weights([v_mult.T])


In [9]:
def custom_loss(y_true, y_pred):
    mask = y_true > 0
    diff = y_true - y_pred
    
    return K.mean(K.square(diff[mask]))
    

In [10]:
mdl.compile(loss=custom_loss,
            optimizer = 'RMSprop')

In [ ]:
mdl.fit(x_one_hot_tr, s_full_tr, epochs=1000, validation_data=(x_one_hot_te, s_full_te))


Epoch 1/1000
53/53 [==============================] - 7s 139ms/step - loss: 11.2107 - val_loss: 8.7769
Epoch 2/1000
53/53 [==============================] - 8s 153ms/step - loss: 4.4741 - val_loss: 9.3757
Epoch 3/1000
53/53 [==============================] - 11s 199ms/step - loss: 3.6175 - val_loss: 9.2595
Epoch 4/1000
53/53 [==============================] - 9s 161ms/step - loss: 3.9382 - val_loss: 9.8067
Epoch 5/1000
53/53 [==============================] - 8s 156ms/step - loss: 3.7083 - val_loss: 9.5769
Epoch 6/1000
53/53 [==============================] - 9s 164ms/step - loss: 3.7294 - val_loss: 9.5805
Epoch 7/1000
53/53 [==============================] - 9s 161ms/step - loss: 3.9789 - val_loss: 9.6836
Epoch 8/1000
53/53 [==============================] - 9s 163ms/step - loss: 3.6627 - val_loss: 9.5352
Epoch 9/1000
53/53 [==============================] - 9s 178ms/step - loss: 4.0041 - val_loss: 9.5377
Epoch 10/1000
53/53 [==============================] - 12s 230ms/step - loss: 3.

In [ ]:
x_oh = np.zeros((1, n_item))
x_oh[0, 0] = 1

mdl.predict(x_oh)